# 重新审视Iris数据集
## To Do:归一化/标准化

In [79]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.optim import Adam

In [80]:
iris_data = load_iris(return_X_y=True)
# print(iris_data)
# print(type(iris_data))
print(type(iris_data[0])) # features input_data
print('*' * 100)
print(iris_data[1])

<class 'numpy.ndarray'>
****************************************************************************************************
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [81]:
print(iris_data[0].shape) # ->[N, Feature_num]

(150, 4)


In [82]:
class IrisDataset(Dataset):

    def __init__(self, X:np.ndarray, y:np.ndarray):
        super(IrisDataset, self).__init__()
        X = X.astype('float32')
        y = y.astype('int64')
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    '''
    ->datas, labels
    '''
    def __getitem__(self, index):
        return X[index], y[index]

    def __len__(self)->int:
        return len(X)

In [83]:
X = iris_data[0]
y = iris_data[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True,random_state=30)
print(X_train)
print(X_train.shape)

[[6.9 3.1 5.4 2.1]
 [5.5 2.4 3.7 1. ]
 [6.8 3.  5.5 2.1]
 [6.1 3.  4.9 1.8]
 [7.7 2.6 6.9 2.3]
 [6.9 3.1 5.1 2.3]
 [5.1 2.5 3.  1.1]
 [6.4 3.1 5.5 1.8]
 [6.1 2.8 4.  1.3]
 [7.2 3.  5.8 1.6]
 [6.8 3.2 5.9 2.3]
 [6.4 2.8 5.6 2.1]
 [5.8 4.  1.2 0.2]
 [6.2 2.2 4.5 1.5]
 [6.4 2.7 5.3 1.9]
 [5.2 2.7 3.9 1.4]
 [5.6 2.7 4.2 1.3]
 [6.1 2.8 4.7 1.2]
 [5.5 2.5 4.  1.3]
 [5.6 3.  4.1 1.3]
 [4.6 3.4 1.4 0.3]
 [5.8 2.7 4.1 1. ]
 [6.7 3.3 5.7 2.5]
 [5.8 2.6 4.  1.2]
 [5.1 3.7 1.5 0.4]
 [6.  2.7 5.1 1.6]
 [4.8 3.  1.4 0.1]
 [5.6 3.  4.5 1.5]
 [6.  3.4 4.5 1.6]
 [6.1 2.9 4.7 1.4]
 [4.7 3.2 1.6 0.2]
 [5.4 3.  4.5 1.5]
 [5.1 3.3 1.7 0.5]
 [5.7 2.9 4.2 1.3]
 [5.4 3.4 1.5 0.4]
 [6.3 3.3 6.  2.5]
 [6.  3.  4.8 1.8]
 [7.2 3.2 6.  1.8]
 [5.3 3.7 1.5 0.2]
 [6.2 3.4 5.4 2.3]
 [6.7 3.  5.  1.7]
 [7.7 2.8 6.7 2. ]
 [5.1 3.5 1.4 0.3]
 [5.4 3.7 1.5 0.2]
 [6.7 3.1 4.7 1.5]
 [5.  3.6 1.4 0.2]
 [6.3 2.5 5.  1.9]
 [6.4 3.2 5.3 2.3]
 [6.7 2.5 5.8 1.8]
 [4.9 2.5 4.5 1.7]
 [5.4 3.9 1.3 0.4]
 [6.3 3.3 4.7 1.6]
 [4.5 2.3 1.

In [84]:
Iris_train_data = IrisDataset(X=X_train, y=y_train)
Iris_test_data = IrisDataset(X=X_test,y=y_test)
print(Iris_train_data)

In [85]:
class Network(nn.Module):
    def __init__(self, feature_num, layer_num, output_num):
        super(Network, self).__init__()
        self.classifier = nn.Sequential()
        layer_num = layer_num
        feature_num = feature_num
        out = feature_num * 4
        for index in range(layer_num - 1):
            self.classifier.add_module(f"Linear{index}", nn.Linear(feature_num, out))
            feature_num = out
            out *= 2
            self.classifier.add_module(f"Relu{index}", nn.ReLU())
        self.classifier.add_module(f'Linear{layer_num - 1}', nn.Linear(feature_num, output_num))
        self.classifier.add_module(f"Relu{layer_num - 1}", nn.ReLU())

    def forward(self, x):
        return self.classifier(x)




In [86]:
net = Network(4,3, 3)
print(net)
opt = Adam(lr=0.01, params=net.parameters())

Network(
  (classifier): Sequential(
    (Linear0): Linear(in_features=4, out_features=16, bias=True)
    (Relu0): ReLU()
    (Linear1): Linear(in_features=16, out_features=32, bias=True)
    (Relu1): ReLU()
    (Linear2): Linear(in_features=32, out_features=3, bias=True)
    (Relu2): ReLU()
  )
)


In [87]:
# To Do:将训练代码包装成训练方法 def train():

net.train()
train_dataloader = DataLoader(Iris_train_data, shuffle=True, batch_size=20)
loss_fn = nn.CrossEntropyLoss()

EPOCH = 200
for epoch in range(EPOCH):
    for batch_id, (input_datas, labels) in enumerate(train_dataloader): # batch_id, data / X, label / y
        input_datas = torch.tensor(input_datas, dtype=torch.float32)
        pred = net(input_datas)
        loss = loss_fn(pred, labels)

        opt.zero_grad()
        loss.backward()
        opt.step()
        print(f"epoch:{epoch}, batch_id:{batch_id}, loss:{loss}")



epoch:0, batch_id:0, loss:1.211965799331665
epoch:0, batch_id:1, loss:1.0686856508255005
epoch:0, batch_id:2, loss:1.0442147254943848
epoch:0, batch_id:3, loss:1.0652010440826416
epoch:0, batch_id:4, loss:1.125232458114624
epoch:0, batch_id:5, loss:1.060730218887329
epoch:0, batch_id:6, loss:0.9012821912765503
epoch:0, batch_id:7, loss:0.9189699292182922
epoch:1, batch_id:0, loss:0.8867731094360352
epoch:1, batch_id:1, loss:0.8482815623283386
epoch:1, batch_id:2, loss:0.7959927320480347
epoch:1, batch_id:3, loss:0.8468713760375977
epoch:1, batch_id:4, loss:0.725127100944519
epoch:1, batch_id:5, loss:0.7068567872047424
epoch:1, batch_id:6, loss:0.6986352205276489
epoch:1, batch_id:7, loss:0.7127848267555237
epoch:2, batch_id:0, loss:0.6438041925430298
epoch:2, batch_id:1, loss:0.5500816702842712
epoch:2, batch_id:2, loss:0.6875663995742798
epoch:2, batch_id:3, loss:0.5314423441886902
epoch:2, batch_id:4, loss:0.5957547426223755
epoch:2, batch_id:5, loss:0.5606933236122131
epoch:2, batch

/var/folders/4q/mxhf2gj12ll_msd5l7shm0dw0000gn/T/ipykernel_5697/2368251147.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_datas = torch.tensor(input_datas, dtype=torch.float32)


epoch:22, batch_id:5, loss:0.09543701261281967
epoch:22, batch_id:6, loss:0.1986825168132782
epoch:22, batch_id:7, loss:0.23044411838054657
epoch:23, batch_id:0, loss:0.12009111791849136
epoch:23, batch_id:1, loss:0.02140304446220398
epoch:23, batch_id:2, loss:0.013396675698459148
epoch:23, batch_id:3, loss:0.1081782728433609
epoch:23, batch_id:4, loss:0.2607097923755646
epoch:23, batch_id:5, loss:0.03799170255661011
epoch:23, batch_id:6, loss:0.01840515248477459
epoch:23, batch_id:7, loss:0.02547835186123848
epoch:24, batch_id:0, loss:0.1539706438779831
epoch:24, batch_id:1, loss:0.020998679101467133
epoch:24, batch_id:2, loss:0.05538841336965561
epoch:24, batch_id:3, loss:0.2231316864490509
epoch:24, batch_id:4, loss:0.09379268437623978
epoch:24, batch_id:5, loss:0.038395803421735764
epoch:24, batch_id:6, loss:0.0041693029925227165
epoch:24, batch_id:7, loss:0.10853558778762817
epoch:25, batch_id:0, loss:0.04859131947159767
epoch:25, batch_id:1, loss:0.050299543887376785
epoch:25, ba

# 评估(evaluate)

In [88]:
net.eval() # batch normalization / dropout ......
test_loader = DataLoader(Iris_test_data, batch_size=10)
prob_trans = nn.Softmax(dim=1)
cnt = 0
with torch.no_grad():
    for batch_id, (Sample_data, labels) in enumerate(test_loader):
        total = labels.shape[0]
        Sample_data = torch.tensor(Sample_data, dtype=torch.float32)
        primary_pred = net(Sample_data) # ->Real value
        prob = prob_trans(primary_pred)
        label_ls = torch.argmax(prob,dim=1)
        for index in range(labels.shape[0]):
            if label_ls[index] == labels[index]:
                cnt += 1
        acc = cnt / total
        cnt = 0
        print(f"bacth_id:{batch_id}, acc:{acc}")

bacth_id:0, acc:1.0
bacth_id:1, acc:1.0
bacth_id:2, acc:1.0
bacth_id:3, acc:1.0
bacth_id:4, acc:1.0
bacth_id:5, acc:1.0
bacth_id:6, acc:1.0
bacth_id:7, acc:0.8
bacth_id:8, acc:0.9
bacth_id:9, acc:1.0
bacth_id:10, acc:1.0
bacth_id:11, acc:1.0
bacth_id:12, acc:1.0
bacth_id:13, acc:1.0
bacth_id:14, acc:1.0


/var/folders/4q/mxhf2gj12ll_msd5l7shm0dw0000gn/T/ipykernel_5697/3079170646.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Sample_data = torch.tensor(Sample_data, dtype=torch.float32)
